In [534]:
#import required modules
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle

%matplotlib inline

In [535]:
#unpack pickled dataframes

modeling = pd.read_pickle("modeling.pkl")
forecast = pd.read_pickle("forecast.pkl")

#change differential into win/loss
modeling["Differential"] = modeling["Differential"].apply(lambda x: 1 if x > 0 else 0)
modeling.head()

,Differential,QB FOR,WR FOR,TE FOR,RB FOR,DT FOR,LT FOR,CB FOR,LB FOR,S FOR,...,DT AGAINST,LT AGAINST,CB AGAINST,LB AGAINST,S AGAINST,RT AGAINST,EDGE AGAINST,C AGAINST,LG AGAINST,RG AGAINST
gamename,,,,,,,,,,,,,,,,,,,,,
Chiefs Texans 2020,1,90.2,84.5,86.1,65.7,87.6,71.2,71.9,50.4,75.4,...,59.8,75.1,64.5,68.6,76.7,59.4,87.0,67.3,59.1,53.4
Seahawks Falcons 2020,1,92.0,81.9,79.6,77.9,68.4,74.1,87.6,76.4,73.4,...,90.1,79.7,56.8,76.6,72.0,53.0,72.0,72.1,87.7,47.0
Bills Jets 2020,1,64.1,78.3,60.2,69.3,65.1,73.1,74.3,65.6,77.7,...,81.3,74.4,79.0,73.9,87.9,60.6,73.1,72.0,56.1,65.6
Raiders Panthers 2020,1,79.9,75.5,83.2,87.1,73.1,64.9,67.4,79.0,69.3,...,83.7,79.3,63.8,65.9,76.4,76.2,54.7,63.4,58.6,63.6
Bears Lions 2020,1,76.6,80.8,58.0,66.6,91.7,58.6,62.5,61.9,66.8,...,85.9,75.5,82.8,72.3,74.4,74.4,83.0,74.9,64.9,78.7


In [536]:
#split modeling data in to test and train (75/25)
X = modeling.drop("Differential", axis = 1)
Y = modeling["Differential"]

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state = 0)

In [537]:
#an AdaBoost Classifier gives nice results with both training and test data
from sklearn.ensemble import AdaBoostClassifier

AB = AdaBoostClassifier().fit(X_train,Y_train)
print(np.sum(np.array(AB.predict(X_test)) == np.array(Y_test))/len(X_test))
print(np.sum(np.array(AB.predict(X_train)) == np.array(Y_train))/len(X_train))

0.6060606060606061
0.7159090909090909


In [538]:
#cross validating
#looks like the the model will be correct ~64% of the time on average
from sklearn.model_selection import cross_val_score
cvscore = cross_val_score(AB, X, Y, cv = 10)
np.average(cvscore)

0.6381401617250673

In [539]:
#applying the model to forecast the 2022 season
AB = AdaBoostClassifier().fit(X,Y)
forecast["result"] = RF.predict(forecast)

In [540]:
#relabel and export to csv
forecast["result"] = forecast["result"].apply(lambda x: "W" if x > 0 else "L")
forecast["result"].to_frame().to_csv("2022predictions.csv")